In [1]:
import random

from hyperopt import hp
from hyperopt.pyll import scope
from ray import train, tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.hyperopt import HyperOptSearch

In [7]:
def train_model(config, input_data):
    factor = random.uniform(0, 1)
    for i in range(input_data["epochs"],1,-1):
        train.report({input_data["metric"]: i*factor})


In [9]:
num_samples = 100
epochs = 100
metric = "mse"

input_data = {
    "epochs": epochs,
    "metric": metric,
}

example_config = {
    "lr_rnvae": 1e-3,
    "lr_imputer": 1e-4,
    "dropout": .1,
    "sm_emb_size": 64,
    "cell_emb_size": 32,
    "latent_dim": 256,
    "hidden_dim": 512,
    "kld_weight": 1,
    "impute_loss_weight": 2,
}

space = {
    "lr_rnvae": hp.loguniform("lr_rnvae", -10, -1),
    "lr_imputer": hp.loguniform("lr_imputer", -10, -1),
    "dropout": hp.uniform("dropout", 0, 1),
    "sm_emb_size": scope.int(hp.qloguniform("sm_emb_size", 0, 3, 1)),
    "cell_emb_size": scope.int(hp.qloguniform("cell_emb_size", 0, 3, 1)),
    "latent_dim": scope.int(hp.qloguniform("latent_dim", 0, 7, 1)),
    "hidden_dim": scope.int(hp.qloguniform("hidden_dim", 0, 7, 1)),
    "kld_weight": hp.loguniform("kld_weight", -2, 2),
    "impute_loss_weight": hp.loguniform("impute_loss_weight", -2, 2),
}

mode = "min"
hyperopt_search = HyperOptSearch(space, metric=metric, mode=mode)
scheduler = ASHAScheduler(metric=metric, grace_period=5, mode=mode, max_t=epochs)
tuner = tune.Tuner(
    tune.with_parameters(train_model, input_data=input_data),
    tune_config=tune.TuneConfig(
        num_samples=num_samples,
        search_alg=hyperopt_search,
        scheduler=scheduler
    ),
    run_config=train.RunConfig(
        failure_config=train.FailureConfig(fail_fast=False))
)
results = tuner.fit()

best_result = results.get_best_result(metric, mode=mode)
print(best_result.path)
print("CONFIG:", best_result.config)
print("METRICS:", best_result.metrics)

2023-10-21 14:28:53,403	INFO tune.py:1143 -- Total run time: 13.53 seconds (12.47 seconds for the tuning loop).


/Users/laurasisson/ray_results/train_model_2023-10-21_14-28-39/train_model_94eae705_60_cell_emb_size=3,dropout=0.7424,hidden_dim=13,impute_loss_weight=0.1813,kld_weight=7.2437,latent_dim=1,lr_i_2023-10-21_14-28-48
CONFIG: {'cell_emb_size': 3, 'dropout': 0.7424053974648346, 'hidden_dim': 13, 'impute_loss_weight': 0.1812690579822345, 'kld_weight': 7.243671545471919, 'latent_dim': 1, 'lr_imputer': 0.004563194716937017, 'lr_rnvae': 0.00011361265198956045, 'sm_emb_size': 5}
METRICS: {'mse': 0.04290977545466945, 'timestamp': 1697912928, 'done': True, 'training_iteration': 99, 'trial_id': '94eae705', 'date': '2023-10-21_14-28-48', 'time_this_iter_s': 0.00016307830810546875, 'time_total_s': 0.02109551429748535, 'pid': 26032, 'hostname': 'Lauras-Air', 'node_ip': '127.0.0.1', 'config': {'cell_emb_size': 3, 'dropout': 0.7424053974648346, 'hidden_dim': 13, 'impute_loss_weight': 0.1812690579822345, 'kld_weight': 7.243671545471919, 'latent_dim': 1, 'lr_imputer': 0.004563194716937017, 'lr_rnvae': 0.0